In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from import_casa import casa

In [3]:
import pandas as pd
import pickle
from gensim.models import FastText
path = casa.get_data_path() / "eda/spm/sp_vectors.model"
model = FastText.load(str(path))
model.wv.vectors.shape

[INFO] 2021-01-19 13:08:27,301 gensim.utils: loading FastText object from E:\LangOn\casa\data\eda\spm\sp_vectors.model
[INFO] 2021-01-19 13:08:28,134 gensim.utils: loading wv recursively from E:\LangOn\casa\data\eda\spm\sp_vectors.model.wv.* with mmap=None
[INFO] 2021-01-19 13:08:28,135 gensim.utils: loading vectors_ngrams from E:\LangOn\casa\data\eda\spm\sp_vectors.model.wv.vectors_ngrams.npy with mmap=None
[INFO] 2021-01-19 13:08:36,130 gensim.utils: setting ignored attribute vectors_ngrams_norm to None
[INFO] 2021-01-19 13:08:36,131 gensim.utils: setting ignored attribute vectors_norm to None
[INFO] 2021-01-19 13:08:36,132 gensim.utils: setting ignored attribute vectors_vocab_norm to None
[INFO] 2021-01-19 13:08:36,132 gensim.utils: setting ignored attribute buckets_word to None
[INFO] 2021-01-19 13:08:36,133 gensim.utils: loading vocabulary recursively from E:\LangOn\casa\data\eda\spm\sp_vectors.model.vocabulary.* with mmap=None
[INFO] 2021-01-19 13:08:36,133 gensim.utils: loading 

(5885, 100)

In [5]:
seeds = pd.read_excel(casa.get_data_path()/"seeded_spm_edited.xlsx")

In [24]:
from casa.topics import SeedEntry, SeedDistribution
seed_entries = []
for _, row in seeds.iterrows():
    candidates = [str(x) for x in row[2:].dropna()]
    entry = SeedEntry(row.Category, row.Seed, candidates)
    seed_entries.append(entry)    

In [ ]:
## collective nouns is ignored: 三雄, 三大電信, 電信三雄
entities_seeds = [
    SeedEntry("電信業者", "中華電信", "中華電信 中華電 中華 種花"),
    SeedEntry("電信業者", "遠傳電信", "遠傳電信 遠傳 遠傳5G"),
    SeedEntry("電信業者", "台灣大哥大", "台灣大哥大 台灣大 台哥大 台哥"),
    SeedEntry("電信業者", "台灣之星", "台灣之星"),
    SeedEntry("電信業者", "亞太", "亞太 亞太電信 亞太電 亞太5G"),    
]

In [27]:
seed_distr = SeedDistribution(model.wv, seed_entries)

In [35]:
seed_distr.get_distribution("中華電信好爛").shape

(880, 4)